In [ ]:
import sys
sys.path.append('src')

In [ ]:
import model
import excel as xlh
import openpyxl as xl
from pprint import pprint

In [ ]:
wbook =xl.load_workbook('config.xlsx', data_only=True)

In [ ]:
arrival_schedule_cancer_df = xlh.get_table(
            wbook, sheet_name='Arrival Schedules', name='ArrivalScheduleCancer').set_index('Hour')
arrival_schedule_noncancer_df = xlh.get_table(
            wbook, sheet_name='Arrival Schedules', name='ArrivalScheduleNonCancer').set_index('Hour')

In [ ]:
arrival_schedules = model.ArrivalSchedules(
    cancer=model.ArrivalSchedule.from_pd(arrival_schedule_cancer_df),
    noncancer=model.ArrivalSchedule.from_pd(arrival_schedule_noncancer_df)
)

In [ ]:
pprint(arrival_schedules.model_dump(), compact=True, width=120)

In [ ]:
xlh.get_table(wbook, sheet_name='Resources', name='Resources').fillna(0.0).set_index('Resource')

In [ ]:
resources_info = {key: model.ResourceInfo(
    name=field.title,
    type=field.json_schema_extra['resource_type'],
    schedule=model.ResourceSchedule.from_pd(
        xlh.get_table(wbook, sheet_name='Resources', name='Resources'
                      ).fillna(0.0).set_index('Resource'),
        row_name=field.title
    )
) for key, field in model.ResourcesInfo.model_fields.items()}
resources_info = model.ResourcesInfo(**resources_info)
pprint(resources_info.model_dump(), compact=True, width=120)

In [ ]:
tasks_df = xlh.get_table(wbook, sheet_name='Task Durations', name='TaskDurations').set_index('Task')

In [ ]:
model.TaskDurationsInfo.model_fields.items()

In [ ]:
task_durations_info = {key: model.DistributionInfo(
    type=tasks_df.loc[field.title,'Distribution'],
    low=tasks_df.loc[field.title,'Optimistic'],
    mode=tasks_df.loc[field.title,'Most Likely'],
    high=tasks_df.loc[field.title,'Pessimistic'],
    time_unit=tasks_df.loc[field.title,'Units'],
) for key, field in model.TaskDurationsInfo.model_fields.items()}
task_durations_info = model.TaskDurationsInfo(**task_durations_info)
pprint(task_durations_info.model_dump(), compact=True, width=120)


In [ ]:
batch_sizes_df = xlh.get_table(wbook, sheet_name='Batch Sizes', name='BatchSizes').set_index('Batch Name')

In [ ]:
{key: batch_sizes_df.loc[field.title, 'Size'] for key, field in model.BatchSizes.model_fields.items()}

In [ ]:
xlh.get_name(wbook, 'NumBlocksLargeSurgical')

In [ ]:
globals_float = {key: xlh.get_name(wbook, field.title) for key, field in model.Globals.model_fields.items() if field.annotation==float}

globals_float

In [ ]:
globals_dists = {key: model.IntDistributionInfo(type=xlh.get_name(wbook, field.title)[0], low=xlh.get_name(wbook, field.title)[1], mode=xlh.get_name(wbook, field.title)[2], high=xlh.get_name(wbook, field.title)[3]) for key, field in model.Globals.model_fields.items() if field.annotation==model.IntDistributionInfo}

globals_dists

In [ ]:
model.Globals(**globals_float, **globals_dists)